In [5]:
import importlib
import torch
import datafawn
importlib.reload(datafawn)

from pathlib import Path

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("CUDA not available, using CPU")


Using GPU: NVIDIA GeForce RTX 5070 Ti Laptop GPU


In [6]:
# =============== POSE ESTIMATOR =============== #
dlc_estimator = datafawn.DeepLabCutPoseEstimator(
    model_name='superanimal_quadruped',
    detector_name='fasterrcnn_resnet50_fpn_v2',
    hrnet_model='hrnet_w32',
    max_individuals=1,
    pcutoff=0.15,
    device=device
)

# =============== POSTPROCESSORS =============== #
rel_paws = ['front_left_paw_rel', 'front_right_paw_rel', 'back_left_paw_rel', 'back_right_paw_rel']
reference_map = {
    'back_base': ['front_left_paw', 'front_right_paw'],
    'tail_base': ['back_left_paw', 'back_right_paw']
}

rel_pp = datafawn.RelativePawPositionPostprocessor()
error_pp = datafawn.ErrorPostprocessor(
    bodyparts=rel_paws,
    use_velocity=True,
    use_likelihood=True,
    use_distance=True,
    velocity_kwargs={'threshold_pixels': 50, 'window_size': 5},
    likelihood_kwargs={'min_likelihood': 0.5},
    distance_kwargs={'reference_map': reference_map, 'max_distance': 300}
)

# =============== EVENT EXTRACTOR =============== #
zeni_extractor = datafawn.ZeniExtractor(
    smooth_window_size=5,
    prominence_percentage=0.05,
    orientation_likelihood_threshold=0.0,
    orientation_smooth_window_size=15,
    show_plots=False
)

# =============== SOUNDSCAPE GENERATOR =============== #
SOUNDSCAPE_CONFIG = {
    'event_sound_map': {
        'front_left_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398494__anthousai__wind-chimes-single-01.wav'),
        'front_right_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398493__anthousai__wind-chimes-single-02.wav'),
        'back_left_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398492__anthousai__wind-chimes-single-03.wav'),
        'back_right_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398496__anthousai__wind-chimes-single-04.wav')
    },
    'backing_track': Path('data_example/sounds/22415__anthousai__wind-chimes/398495__anthousai__wind-chimes-rough.wav'),
    'volume': {
        'backing_track': 0.10,
        'event_sounds': 0.65,
        'original_video': 4.0
    }
}
ss_generator = datafawn.SoundScapeFromConfig(soundscape_config=SOUNDSCAPE_CONFIG)

In [7]:
# Create the FULL pipeline with all components
pipeline = datafawn.EventDetectionPipeline(
    pose_estimator=dlc_estimator,
    postprocessors=[rel_pp, error_pp],
    event_extractors=[zeni_extractor],
    soundscape_generators=[ss_generator]
)

In [8]:
# Run full pipeline from raw video
RAW_VIDEO_PATH = 'data/raw_videos/spike_elk.mp4'
OUTPUT_DIR = 'data/spike_elk'

results = pipeline.run(
    video_path=RAW_VIDEO_PATH,
    output_dir=OUTPUT_DIR,
    soundscape_input_video="pose_est"  # Use the labeled video for soundscape
)

Loading DLC 3.0.0rc13...
Running video inference on [WindowsPath('data/raw_videos/spike_elk.mp4')] with superanimal_quadruped_hrnet_w32
Using pytorch for model hrnet_w32
Processing video data\raw_videos\spike_elk.mp4
Starting to analyze data\raw_videos\spike_elk.mp4
Video metadata: 
  Overall # of frames:    600
  Duration of video [s]:  20.00
  fps:                    30.0
  resolution:             w=1296, h=720

Running detector with batch size 1


100%|██████████| 600/600 [00:43<00:00, 13.66it/s]


Running pose prediction with batch size 1


100%|██████████| 600/600 [00:32<00:00, 18.49it/s]


Saving results to data\spike_elk\pose_estimation
Saving results in data\spike_elk\pose_estimation\spike_elk_superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_.h5 and data\spike_elk\pose_estimation\spike_elk_superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2__full.pickle


...

Duration of video [s]: 20.0, recorded with 30.0 fps!
Overall # of frames: 600 with cropped frame dimensions: 1296 720
Generating frames and creating video.


100%|██████████| 600/600 [00:09<00:00, 63.46it/s]


Video with predictions was saved as data\spike_elk\pose_estimation
📁 Saved pose data: data\spike_elk\pose_estimation\pose_data.h5
Dataframe with relative positions created:
Original shape: (600, 117)
New shape: (600, 129)

New bodyparts added:
['back_left_paw_rel', 'back_right_paw_rel', 'front_left_paw_rel', 'front_right_paw_rel']
📁 Saved postprocessed data: data\spike_elk\postprocessing\postprocessed_data.h5
📁 Saved events: data\spike_elk\events\events.json
Loading video: data\spike_elk\pose_estimation\spike_elk_superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2__labeled_before_adapt.mp4
Video FPS: 30.0
Video duration: 20.00 seconds

Processing individual: animal0 (scorer: superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_)

  Processing front_right_paw_strike:
    Found 6 strikes
    Strike timestamps: ['1.57s', '3.27s', '5.00s']...
    Sound duration: 3.16 seconds
    Created 6 audio clips for front_right_paw_strike

  Processing back_left_paw_strike:
    Found 9 st

MoviePy - Done.
MoviePy - Writing video data\spike_elk\soundscapes\SoundScapeFromConfig_output.mp4



MoviePy - Done !
MoviePy - video ready data\spike_elk\soundscapes\SoundScapeFromConfig_output.mp4
Done! Output saved to: data\spike_elk\soundscapes\SoundScapeFromConfig_output.mp4
📁 Saved soundscape: data\spike_elk\soundscapes\SoundScapeFromConfig_output.mp4


In [ ]:
# Partial pipeline after events are extracted
results = pipeline.run(
    events_path='data/spike_elk/events/events.json',
    output_dir='data/soundscape_tests/',
    soundscape_input_video='data/spike_elk/pose_estimation/spike_elk_superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2__labeled_before_adapt.mp4'
)